# IMPORTS

In [1]:
import re
import math
import sqlite3
import requests
import pandas as pd
import numpy  as np

from datetime   import datetime
from bs4        import BeautifulSoup
from sqlalchemy import create_engine



# DATA COLLECT

In [2]:

#PARAMENTS
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}


#link princial do modelo de roupa que pŕeciso
url= 'https://www2.hm.com/en_us/men/products/jeans.html'

#REQUEST TO URL
page = requests.get( url, headers=headers )


#BEAUTIFUL OBJECT
soup = BeautifulSoup(page.text, 'html.parser')

# ================================ PRODUCT DATA =========== # 

products = soup.find( 'ul', class_='products-listing small' )


#list of products
products_list= products.find_all ('article',class_="hm-product-item")


#product id

product_id = [p.get('data-articlecode') for p in products_list]

#product category

products_category= [p.get('data-category') for p in products_list]

#product name 

#extraction all the name 
products_list_2=  products.find_all('a', class_='link')

products_name =[p.get_text('link') for p in products_list_2]

#product price

#extraction informations of price
products_list_3 = products.find_all('span', class_='price regular')

#find all the price in page
products_price= [p.get_text('price_regular') for p in products_list_3]

data = pd.DataFrame([product_id,products_category,products_name,products_price]).T

data.columns = ['product_id','products_category','products_name','products_price']

data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d-%H-%M:%S')

In [3]:
data.shape

(36, 5)

In [13]:
data.head()

,product_id,products_category,products_name,products_price,scrapy_datetime
0,0985159001,men_jeans_skinny,Skinny Jeans,$ 19.99,2022-05-15-13-47:53
1,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-05-15-13-47:53
2,1024256008,men_jeans_slim,Slim Jeans,$ 19.99,2022-05-15-13-47:53
3,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-05-15-13-47:53
4,1008549006,men_jeans_regular,Regular Jeans,$ 19.99,2022-05-15-13-47:53


# DATA  COLLECT BY PRODUCT

In [4]:
#key browser
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}

#empty Data Frame (to insert the joins)
df_compositions= pd.DataFrame()

aux= []

cols= ['Composition','Size','Art. No.','Fit','messages.garmentLength','messages.waistRise',
'Care instructions',
 'Material',
 'Description',
 'Imported',
 'Concept',
 'Nice to know']
    
df_pattern = pd.DataFrame( columns=  cols )


#API Requests
for i in range(len(data)):
    
    url= 'https://www2.hm.com/en_us/productpage.'+ data.loc[i,'product_id'] +'.html'
    #url= 'https://www2.hm.com/en_us/productpage.'+ '1024256001' +'.html'
    
    
    
    
    page = requests.get( url, headers=headers )

    #Beautifull Souo object
    soup= BeautifulSoup(page.text,'html.parser')

    ### ===================== color name ======================

    products_list= soup.find_all('a',class_='filter-option miniature active') + soup.find_all('a',class_='filter-option miniature') 

    #product id
    products_id = [p.get('data-articlecode') for p in products_list]

    #color name
    color_name = [p.get('data-color') for p in products_list]

    #create data frame

    df_color = pd.DataFrame([products_id,color_name]).T

    #rename the columns of DF
    df_color.columns= ['Art. No.','color_name']
    
    
    for j in range(len(df_color)):
        url2= 'https://www2.hm.com/en_us/productpage.'+ df_color.loc[j,'Art. No.'] +'.html'
        #print( 'Color: {}'.format( url2 ) )
        
    
        page = requests.get( url2, headers=headers )

        #Beautifull Souo object
        soup= BeautifulSoup(page.text,'html.parser')
        
        #=========================product name ==============================
        product_name = soup.find_all('h1',class_='primary product-item-headline')
        product_name = product_name[0].get_text()
        
        # ==================== product price ======================= #
        
        product_price= soup.find_all('div',class_='primary-row product-item-price')
        product_price = re.findall(r'\d+?.\d+',product_price[0].get_text())[0]
       

        # ============================== composition =============================== #

        #composition
        composition_list= soup.find_all('div', class_ = 'details-attributes-list-item')
        
        #composition_list = soup.find_all('dd','details-list-item')
        #product_composition = composition_list[3]

        product_composition = [list(filter (None, p.get_text().split('\n') ) ) for p in composition_list]
        

        if product_composition != []:

            #created the dataframe aux
            df_composition= pd.DataFrame(product_composition).T
        
            
            #renome the columns
            df_composition.columns= df_composition.iloc[0]

            #delete the first row and delete the columns that is the empty
            df_composition= df_composition.iloc[1:].fillna(method='ffill')
            

            #remove pocket lining, shell and lining
            df_composition['Composition'] = df_composition['Composition'].replace('Pocket lining:','',regex=True)
            df_composition['Composition'] = df_composition['Composition'].replace('Shell:','',regex=True)
            df_composition['Composition'] = df_composition['Composition'].replace('Lining:','',regex=True)
            df_composition['Composition'] = df_composition['Composition'].replace('Pocket: Cotton 100%','',regex=True)
        
            #Pocket: Cotton 100%',
    
            # garantee the same number of columns
            df_composition = pd.concat( [df_pattern, df_composition], axis=0 )
            
            #product name 
            df_composition['product_name'] = product_name
            df_composition['product_price'] = product_price
            

            #keep new columns if it shows
            aux= aux + df_composition.columns.tolist()

    
            #merge dataframe of color and composition 
            df_composition= pd.merge(df_composition,df_color, how='left', on ='Art. No.')
            


            #all products 
            df_compositions = pd.concat([df_composition,df_compositions], axis=0)

        
        
    else:
        None

#select just columns necessary
df_compositions= df_compositions[['Art. No.','product_name','product_price','Composition','color_name','Size','Fit',]]

#rename columns
df_compositions.rename(columns = {'Art. No.':'product_id','Composition':'composition','Size':'size',
                                  'Fit':'fit'},inplace=True)

# Join Showroom data + details

df_compositions['stlye_id'] = df_compositions['product_id'].apply( lambda x: x[:-3] )
df_compositions['color_id'] = df_compositions['product_id'].apply( lambda x: x[-3:] )

df_compositions['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d-%H-%M:%S')



In [9]:
df_compositions.head()

,product_id,product_name,product_price,composition,color_name,size,fit,stlye_id,color_id,scrapy_datetime
0,1013317008,\n\t\t\t\t\t\t\t Hybrid Regular Tapered Joggers,39.99,"Cotton 77%, Polyester 21%, Spandex 2%",Dark blue,NaN,Regular fit,1013317,008,2022-05-19-16-03:17
1,1013317008,\n\t\t\t\t\t\t\t Hybrid Regular Tapered Joggers,39.99,"Cotton 77%, Polyester 21%, Spandex 2%",Dark blue,NaN,Regular fit,1013317,008,2022-05-19-16-03:17
2,1013317008,\n\t\t\t\t\t\t\t Hybrid Regular Tapered Joggers,39.99,"Cotton 77%, Polyester 21%, Spandex 2%",Dark blue,NaN,Regular fit,1013317,008,2022-05-19-16-03:17
3,1013317008,\n\t\t\t\t\t\t\t Hybrid Regular Tapered Joggers,39.99,"Cotton 77%, Polyester 21%, Spandex 2%",Dark blue,NaN,Regular fit,1013317,008,2022-05-19-16-03:17
4,1013317008,\n\t\t\t\t\t\t\t Hybrid Regular Tapered Joggers,39.99,"Cotton 77%, Polyester 21%, Spandex 2%",Dark blue,NaN,Regular fit,1013317,008,2022-05-19-16-03:17


array([False])

In [165]:
len(df_compositions['product_id'].unique())

106

# DATA CLEANING

In [10]:
#product id
df_data = df_compositions.dropna(subset=['product_id'])

#product category

#product name
df_data['product_name']= df_data['product_name'].str.replace('\n','')
df_data['product_name']= df_data['product_name'].str.replace('\t','')
df_data['product_name']= df_data['product_name'].str.replace('  ','')
df_data['product_name']= df_data['product_name'].str.replace(' ','_').str.lower()

#products_price

df_data['product_price'] = df_data['product_price'].astype(float)

# Fit
df_data['fit'] = df_data['fit'].str.replace(' ','_').str.lower()

#product datetime

#data['scrapy_datetime']= pd.to_datetime(data['scrapy_datetime'])


#color name
df_data['color_name']= df_data['color_name'].str.replace(' ','_').str.lower()

#break composition by comma (separando as composições dos materiais por coluna)

df1 = df_data['composition'].str.split(',',expand=True).reset_index(drop=True)

#Cotton| Spandex | Polyester |  Elastomultiester
#2 etapa
#definir as colunas (organizando as colunas por máteria-prima)

#criando Dataframe vazio com o mesmo tamanho do dataframe original para prencher com as respectivas composições
df_ref = pd.DataFrame(index=np.arange(len(df_data) ),columns=['cotton','spandex','polyester','elastomultiester'])

#================================ Composition =======================================#

#cotton
#pegando a coluna da composição cotton

#coletando cotton da primeira coluna df1
df_cotton_0 = df1.loc[df1[0].str.contains('Cotton',na=True),0]
#rename colum
df_cotton_0.name = 'cotton'

#coletando cotton da segunda coluna df1
df_cotton_1 = df1.loc[df1[1].str.contains('Cotton',na=True),1]
#rename columns
df_cotton_1.name ='cotton'

#combine the columns cotton
df_cotton = df_cotton_0.combine_first(df_cotton_1)

#concat tables
df_ref = pd.concat([df_ref,df_cotton],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

###================== polyester ================= ##

df_polyester_0 = df1.loc[df1[0].str.contains('Polyester',na=True),0]
df_polyester_0.name = 'polyester'

df_polyester_1 = df1.loc[df1[1].str.contains('Polyester',na=True),1]
df_polyester_1.name = 'polyester'

#combine the columns cotton
df_polyester = df_polyester_0.combine_first(df_polyester_1)

#concat tables
df_ref = pd.concat([df_ref,df_polyester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

# =============================== Elastomultiester ============================== #

df_elastomultiester = df1.loc[df1[1].str.contains('Elastomultiester',na=True),1]
#rename colum
df_elastomultiester.name = 'elastomultiester'

#concat tables
df_ref = pd.concat([df_ref,df_elastomultiester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]


# ======================== Spandex ========================================= #

df_spandex_1 = df1.loc[df1[1].str.contains('Spandex', na=True),1]
df_spandex_1.name = 'spandex'

df_spandex_2= df1.loc[df1[2].str.contains('Spandex', na=True),2]
df_spandex_2.name = 'spandex'

df_spandex = df_spandex_1.combine_first(df_spandex_2)

#concat tables
df_ref = pd.concat([df_ref,df_spandex],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]


#join of combine with product_id
df_aux= pd.concat([df_data['product_id'].reset_index(drop=True),df_ref.reset_index(drop=True)],axis=1)

#format composition data

#cotton
df_aux['cotton']= df_aux['cotton'].apply(lambda x: int(re.search('\d+',x ).group(0))/100 if pd.notnull(x) else x)

#spandex
df_aux['spandex'] =df_aux['spandex'].apply(lambda x: int(re.search('\d+', x).group(0)) /100 if pd.notnull(x) else x)

#polyester
df_aux['polyester']= df_aux['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

#elastomultiester
df_aux['elastomultiester']=df_aux['elastomultiester'].apply(lambda x: int(re.search('\d+', x).group(0)) /100 if pd.notnull(x) else x)


#======================== final join ==================#


#escolhendo o produto com maior composição para fazer o join
df_aux= df_aux.groupby('product_id').max().reset_index().fillna(0)


#merge the both columns
df_data = pd.merge(df_data,df_aux, on='product_id', how='left')

#drop columns
df_data= df_data.drop('composition',axis=1)

#drop duplicates

df_data= df_data.drop_duplicates()

df_data.shape

(83, 13)

In [12]:
df_data.head()

,product_id,product_name,product_price,color_name,size,fit,stlye_id,color_id,scrapy_datetime,cotton,polyester,elastomultiester,spandex
0,1013317008,hybrid_regular_tapered_joggers,39.99,dark_blue,NaN,regular_fit,1013317,008,2022-05-19-16-03:17,0.77,0.21,0.0,0.02
6,1013317006,hybrid_regular_tapered_joggers,39.99,black,NaN,regular_fit,1013317,006,2022-05-19-16-03:17,0.79,0.20,0.0,0.01
12,1013317005,hybrid_regular_tapered_joggers,39.99,dark_gray,NaN,regular_fit,1013317,005,2022-05-19-16-03:17,0.79,0.19,0.0,0.02
18,1013317004,hybrid_regular_tapered_joggers,20.99,black,NaN,regular_fit,1013317,004,2022-05-19-16-03:17,0.78,0.21,0.0,0.01
24,1013317002,hybrid_regular_tapered_joggers,39.99,denim_blue,NaN,regular_fit,1013317,002,2022-05-19-16-03:17,0.78,0.21,0.0,0.01


In [18]:
df_data.columns

Index(['product_id', 'product_name', 'product_price', 'color_name', 'size',
       'fit', 'stlye_id', 'color_id', 'scrapy_datetime', 'cotton', 'polyester',
       'elastomultiester', 'spandex'],
      dtype='object')

# DATA INSERT

In [19]:
#copy the datas
data_insert = df_data.copy(deep=True)

In [24]:
#montando e formatando os dados que serao armazenado na tabela do SQl

#query_showroom_schema = """ 
                   # CREATE TABLE vitrine(
                   # product_id          TEXT,
                   # product_name        TEXT,
                   # product_price       REAL,
                   # color_name          TEXT,
                   # size                TEXT,
                   # fit                 TEXT,
                   # stlye_id            TEXT,
                   # color_id            TEXT,
                   # cotton              REAL,
                   # polyester           REAL,
                   # elastomultiester    REAL,
                   # spandex             REAL,
                   # scrapy_datetime     TEXT
                    
                    
               # )
#"""

In [25]:
#create table 

conn = sqlite3.connect('database_hm.sqlite')
cursor = conn.execute(query_showroom_schema)
conn.commit()

In [27]:
#create database connection
create_engine('sqlite:///database_hm_sqlite',echo=False)

#insert data
data_insert.to_sql('vitrine',con=conn, if_exists='append', index=False)

In [34]:
#verificando e lendo o arquivo dentro do ambiente SQL

query = """ 
        SELECT * FROM vitrine
        

"""

df = pd.read_sql_query(query,conn)

df.head(50)

,product_id,product_name,product_price,color_name,size,fit,stlye_id,color_id,cotton,polyester,elastomultiester,spandex,scrapy_datetime
0,1013317008,hybrid_regular_tapered_joggers,39.99,dark_blue,None,regular_fit,1013317,008,0.77,0.21,0.00,0.02,2022-05-19-16-03:17
1,1013317006,hybrid_regular_tapered_joggers,39.99,black,None,regular_fit,1013317,006,0.79,0.20,0.00,0.01,2022-05-19-16-03:17
2,1013317005,hybrid_regular_tapered_joggers,39.99,dark_gray,None,regular_fit,1013317,005,0.79,0.19,0.00,0.02,2022-05-19-16-03:17
3,1013317004,hybrid_regular_tapered_joggers,20.99,black,None,regular_fit,1013317,004,0.78,0.21,0.00,0.01,2022-05-19-16-03:17
4,1013317002,hybrid_regular_tapered_joggers,39.99,denim_blue,None,regular_fit,1013317,002,0.78,0.21,0.00,0.01,2022-05-19-16-03:17
5,1013317001,hybrid_regular_tapered_joggers,39.99,light_denim_blue,None,regular_fit,1013317,001,0.77,0.21,0.00,0.02,2022-05-19-16-03:17
6,1013317010,hybrid_regular_tapered_joggers,39.99,black,None,regular_fit,1013317,010,0.80,0.19,0.00,0.01,2022-05-19-16-03:17
7,1004199004,skinny_cropped_jeans,29.99,white,None,skinny_fit,1004199,004,0.99,0.65,0.00,0.01,2022-05-19-16-03:17
8,1004199003,skinny_cropped_jeans,29.99,denim_blue,None,skinny_fit,1004199,003,0.99,0.65,0.00,0.01,2022-05-19-16-03:17
9,1004199002,skinny_cropped_jeans,29.99,black,None,skinny_fit,1004199,002,0.99,0.00,0.00,0.01,2022-05-19-16-03:17
